# Connect to Microsoft SQL Server using PySpark
This notebook demonstrates how to connect to a Microsoft SQL Server database from PySpark using the JDBC driver.

In [38]:
# Step 1: Import necessary libraries
from pyspark.sql import SparkSession

In [39]:
!ls ./jars/mssql-jdbc-12.10.0.jre11.jar

./jars/mssql-jdbc-12.10.0.jre11.jar


In [40]:
!ls -lsat

total 120
 0 drwxrwxrwx 1 root   root    512 May 23 23:00 .
12 -rw-r--r-- 1 jovyan users  9212 May 23 23:00 7_pyspark_mssql_connection.ipynb
 0 -rw-r--r-- 1 jovyan users   123 May 23 22:56 .env
 0 drwxr-xr-x 1 jovyan users   512 May 23 22:55 .ipynb_checkpoints
 0 drwxr-xr-x 1 jovyan users   512 May 23 22:13 jars
 8 -rw-r--r-- 1 jovyan users  6066 May 23 13:47 1_Intro_pyspark_code_part_1.ipynb
 8 drwsrws--- 1 jovyan users  4096 May 23 12:45 ..
20 -rw-r--r-- 1 jovyan users 19268 May 17 02:05 3_Filter_multiple_conditions.ipynb
 8 -rw-r--r-- 1 jovyan users  6062 May 17 01:59 5_essential_pyspark_cheatsheet.ipynb
12 -rw-r--r-- 1 jovyan users  9130 May 17 01:45 2_Intro_pyspark_code_part_2.ipynb
 0 drwxr-xr-x 1 jovyan users   512 May 17 01:31 my_parquet_file.parquet
 4 -rw-r--r-- 1 jovyan users  2649 May 17 00:29 why_pyspark.ipynb
 8 -rw-r--r-- 1 jovyan users  5329 May 17 00:26 why_spark_vs_pandas.ipynb
12 -rw-r--r-- 1 jovyan users 11564 May 16 21:35 6_pyspark_examples_essentials.ipynb
 8 -rw-

In [41]:
!pip install python-dotenv

In [42]:
import os
from dotenv import load_dotenv

In [43]:
# Step 2: Create a Spark session and specify the JDBC driver path
spark = SparkSession.builder \
    .appName("MSSQL Connection") \
    .config("spark.jars", "./jars/mssql-jdbc-12.10.0.jre11.jar") \
    .getOrCreate()

In [44]:
# Load environment variables
load_dotenv()

# Retrieve secrets
host = os.getenv("host")
port = os.getenv("port")
username = os.getenv("username")
password = os.getenv("password")
database = os.getenv("database")

In [45]:
# Step 3: Define the JDBC URL and connection properties
jdbc_url = f"jdbc:sqlserver://{host}:{port};databaseName={database};encrypt=true"

jdbc_url = f"jdbc:sqlserver://{host}:{port};databaseName={database};encrypt=true;trustServerCertificate=true"
    

connection_properties = {
    "user": username,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [46]:
# Step 4: Read data from SQL Server table into a DataFrame
df = spark.read.jdbc(url=jdbc_url, table="factura", properties=connection_properties)
df.show()

+----------+--------+-------------------+-------+
|     Fecha|Producto|              Lugar| Precio|
+----------+--------+-------------------+-------+
|2023-01-01|  azucar|perimercados escazu|1998.75|
|2023-01-01|frijoles|     pali santa ana|1999.53|
|2023-01-01|    atun|automercado heredia| 901.14|
|2023-01-01|    jugo|automercado heredia|  497.8|
|2023-01-01|  pepino|     pali santa ana|1498.21|
|2023-01-01|  pepino|perimercados escazu|1501.71|
|2023-01-01|   leche|perimercados escazu| 797.18|
|2023-01-01|   queso|    masxmenos belen|2004.48|
|2023-01-01|  tomate|perimercados escazu|2503.08|
|2023-01-01|  azucar|automercado heredia|2002.03|
|2023-01-01|  tomate|perimercados escazu|2502.59|
|2023-01-01|    atun|     pali santa ana| 901.23|
|2023-01-01|     pan|automercado heredia|1499.88|
|2023-01-01|   leche|perimercados escazu| 800.86|
|2023-01-01|    jugo|    masxmenos belen| 499.63|
|2023-01-01|frijoles|    masxmenos belen|2000.42|
|2023-01-01|   queso|automercado heredia|2000.02|


In [47]:
# Step 5: Optional - Write DataFrame back to SQL Server
df.write.jdbc(url=jdbc_url, table="nueva_factura", mode="overwrite", properties=connection_properties)